<a href="https://colab.research.google.com/github/s00hyun/Today-I-Learned/blob/master/ML-DL/deep-learning-pytorch/lec4_cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 라이브러리 로드

In [0]:
import torch, torchvision

In [0]:
import torch.nn as nn # 딥러닝을 위한 torch 함수 제공
import torch.nn.functional as F  # 딥러닝에 유용한 수학 함수 제공
import torchvision.transforms as transforms  # 이미지 변환에 유용한 함수 제공

## 2. 모델 선언

In [0]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):  # 신경망이 분류할 이미지 항목 갯수 지정
    super(ConvNet, self).__init__()

    ### nn.Sequential
    # in_channels: 입력 채널 수 (흑백=>1, 컬러=>3)
    # out_channels: 출력 채널 수
    # kernel_size: convolution filter의 가로세로 크기
    # padding: zero padding의 두께

    ### nn.MaxPool2d
    # kernel_size: maxpool size. 
    # stride: 일반적으로 kernel_size와 동일한 값을 사용
    #         maxpooling 수행 시 window가 건너뛰게 되는 기본 길이

    ### nn.Linear(in_features(입력 데이터 개수), out_channel(출력 데이터 개수))
    # 중요: 출력 데이터의 개수 == 클래스 개수 (num_classes)
    self.layer1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=16, 
                                          kernel_size=5, stride=1, padding=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=32, 
                                          kernel_size=5, stride=1, padding=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc = nn.Linear(7 * 7 * 32, num_classes)
  
  # 연산 순서를 정의
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)

    # 텐서를 벡터 형태로 편다. 목표로 하는 텐서 사이즈를 넣어준다. 
    # ex. (batch_size, 32, 7, 7) => (batch_size, 1568)
    flatten = out.reshape(out.size(0), -1)  

    score = self.fc(flatten)
    prob = F.softmax(score, dim=1)

    return prob

## 3. 조건 설정

In [0]:
num_epochs, num_classes, batch_size, learning_rate = 10, 10, 100, 0.1

## 4. 데이터로더 설정

In [0]:
# root: 데이터셋 저장 위치
# train: train dataset인지 아닌지를 결정
# transforms.ToTensor(): 이미지 행렬을 PyTorch 텐서로 변환
# download: 데이터셋 다운로드 여부
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor())

# torch.utils.data.DataLoader(): MNIST 데이터셋을 입력으로 받아 Batch Size만큼 묶어줌
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!





## 5. 모델 설정

In [0]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 6. Training 수행

In [0]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # 입력 이미지를 model에 입력하여 예측값 산출
    outputs = model(images)
    # 예측값과 정답 레이블 간의 차이 계산
    loss = criterion(outputs, labels)
    # 매 data batch마다 optimizer 내의 gradient 값을 모두 0으로 초기화
    optimizer.zero_grad()
    # loss (예측값과 정답 간의 차이) 를 통해서 gradient를 계산
    loss.backward()
    # 계산된 gradient값을 이용하여 모델 내의 변수를 업데이트
    optimizer.step()
    # 예측 레이블과 정답 레이블을 이용해 정확도 산출
    # torch.max: torch tensor 내에서 가장 높은 값과, 그 값의 index를 추출함
    _, predicted = torch.max(outputs.data, 1)  # dimension=1

    correct = (predicted == labels).sum().item()  # 텐서 요소가 1개일 경우 item()으로 값을 얻을 수 있음
    if (i + 1) % 100 == 0:
      print('Epoch: {}/{}, Batch Step: {}/{}, Loss: {:.4f}, Training Accuracy of the Current Batch: {}%'.
                  format(epoch + 1, num_epochs, i + 1, train_loader.__len__(), loss.item(), 100 * correct / batch_size))
      

Epoch: 1/10, Batch Step: 100/600, Loss: 2.1383, Training Accuracy of the Current Batch: 39.0%
Epoch: 1/10, Batch Step: 200/600, Loss: 1.8610, Training Accuracy of the Current Batch: 61.0%
Epoch: 1/10, Batch Step: 300/600, Loss: 1.8435, Training Accuracy of the Current Batch: 62.0%
Epoch: 1/10, Batch Step: 400/600, Loss: 1.8092, Training Accuracy of the Current Batch: 65.0%
Epoch: 1/10, Batch Step: 500/600, Loss: 1.7833, Training Accuracy of the Current Batch: 68.0%
Epoch: 1/10, Batch Step: 600/600, Loss: 1.6585, Training Accuracy of the Current Batch: 81.0%
Epoch: 2/10, Batch Step: 100/600, Loss: 1.6214, Training Accuracy of the Current Batch: 87.0%
Epoch: 2/10, Batch Step: 200/600, Loss: 1.6553, Training Accuracy of the Current Batch: 82.0%
Epoch: 2/10, Batch Step: 300/600, Loss: 1.5730, Training Accuracy of the Current Batch: 91.0%
Epoch: 2/10, Batch Step: 400/600, Loss: 1.6549, Training Accuracy of the Current Batch: 79.0%
Epoch: 2/10, Batch Step: 500/600, Loss: 1.6592, Training Acc

## 7. Test 수행

In [0]:
# 모델을 학습 모드로 변경 (모델 변수 고정)
model.eval()

with torch.no_grad():  # 테스트 과정에서 gradient 계산을 배제
  total, correct = 0, 0  # 전체 데이터 개수, 맞춘 데이터 개수
  for images, lables in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)  # values, indices
    total += labels.size(0)
    correct += (predicted == labels).sum().item()  # True의 갯수를 더한 후 텐서의 성분 추출
  print('Test Accuracy of the 10,000 Test Images: {}%'.format(100 * correct / total))


Test Accuracy of the 10,000 Test Images: 10.12%


## 8. Model 저장

In [0]:
# model의 parameter 저장

# model.state_dict(): 각 레이어에 해당하는 학습 가능한 parameter를 딕셔너리 형태로 나타낸 것
# 이 학습된 모델의 정보를 파이토치 파일로 저장 (확장자: .pt 또는 .pth)
torch.save(model.state_dict(), 'cnn.pt')

In [0]:
!ls

cnn.pt	data  drive  gdrive  sample_data


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
!ls gdrive/'My Drive'/'Colab Notebooks'

딥러닝수업  모두를위한딥러닝  Deep-learning-with-python  Untitled0.ipynb


In [0]:
!mv cnn.pt gdrive/'My Drive'/'Colab Notebooks'/'딥러닝수업'

In [0]:
!ls gdrive/'My Drive'/'Colab Notebooks'/'딥러닝수업'

 cnn.pt						  HW01.ipynb
'(Exp) 01_Python and NumPy basics.ipynb의 사본'   lec4-cnn-mnist
'(Exp) 02_Pytorch Basics.ipynb의 사본'		  lec4-cnn-mnist-plot.ipynb
